# Normal Traning

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '0'  # Chọn GPU đầu tiên

In [2]:
import sys
sys.path.append('/home/jupyter-iec_roadquality/Security/1iat/DropoutAttack/PREVIOUS KD with Resnet50 - Resnet10/modules')
from custom_dropout import DeterministicDropout
from model_wrapper import NetWrapper
from import_data import load_cifar100
from misc import write_to_json
from torch import nn, optim
import torch
import pandas as pd
import torchvision.transforms as transforms
from sklearn.metrics import f1_score, accuracy_score
from os.path import exists
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
import os
import json
from torch import nn
from torchvision.models import vgg16, mobilenet_v2

/home/jupyter-iec_roadquality/.local/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
class TeacherNet(nn.Module):
    def __init__(self, dropout):
        """
        VGG16-based teacher model with customizable dropout.

            Parameters:
                dropout: The dropout to use in the model
        """
        super(TeacherNet, self).__init__()
        self.dropout = dropout

        # Load the VGG16 model
        self.vgg = vgg16(weights='IMAGENET1K_V1')

        for param in self.vgg.features.parameters():
            param.requires_grad = False

        self.vgg.classifier = nn.Sequential(
            nn.Linear(25088, 4096),
            nn.ReLU(True),
            self.dropout,
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            self.dropout,
            nn.Linear(4096, 100)  # CIFAR-10 has 10 classes
        )

    def forward(self, x):
        return self.vgg(x)


class StudentNet(nn.Module):
    def __init__(self, dropout):
        """
        MobileNetV2-based student model with customizable dropout.

            Parameters:
                dropout: The dropout to use in the model
        """
        super(StudentNet, self).__init__()
        self.dropout = dropout

        # Load the MobileNetV2 model
        self.mobilenet = mobilenet_v2(pretrained=False)

        # Replace the classifier
        self.mobilenet.classifier = nn.Sequential(
            nn.Linear(self.mobilenet.last_channel, 256),
            nn.ReLU(),
            self.dropout,
            nn.Linear(256, 100)
        )

    def forward(self, x):
        return self.mobilenet(x)



In [4]:
# Softening by dividing by temperature.
ce_loss = nn.CrossEntropyLoss()
def distillation_loss(student_logits, teacher_logits, labels, temperature, alpha):
    soft_teacher_targets = nn.functional.softmax(teacher_logits / temperature, dim=-1)
    soft_student_probs = nn.functional.log_softmax(student_logits / temperature, dim=-1)     
    kl_divergence_loss = torch.sum(soft_teacher_targets * (soft_teacher_targets.log() - soft_student_probs)) / soft_student_probs.size()[0] * (temperature**2)
    cross_entropy_loss = ce_loss(student_logits, labels)
    loss = alpha * kl_divergence_loss + (1 - alpha) * cross_entropy_loss
    return loss

In [5]:
batch_size = 128
classes = list(range(100))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
transform = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
                             std=[0.2023, 0.1994, 0.2010])
    ])
_, _, _, trainloader, validationloader, testloader = load_cifar100(batch_size, transform)
teacher_model = TeacherNet(nn.Dropout(0.5)).to(device)
teacher_wrapper = NetWrapper(teacher_model, nn.CrossEntropyLoss(), optim.Adam, [0.0001, (0.9, 0.999), 1e-8, 1e-6])
#teacher_wrapper.fit(trainloader, validationloader, 50, verbose=True, num_classes=100)

Files already downloaded and verified
Files already downloaded and verified


In [6]:
torch.save(teacher_model.state_dict(), "../new_output/VGG16-MobileNetV2-CIFAR100/normal-training/vgg16-mobile2-cifar100-teacher_best.pth")

In [7]:
classes = classes = list(range(100))
output_path = f'../new_output/VGG16-MobileNetV2-CIFAR100/normal-training/vgg16-mobileV2-cifar100-teacher_best.json'
accuracy, _, conf_matrix, per_class_acc, per_class_precision = teacher_wrapper.evaluate(testloader,num_classes = 100)
write_to_json(
    output_path,
    'model',
    teacher_wrapper,
    accuracy,
    conf_matrix,
    per_class_acc,
    per_class_precision,
    classes
)

In [8]:
teacher_dropout = nn.Dropout(p=0.5)
teacher_model = TeacherNet(teacher_dropout).to(device)
#teacher_optimizer = optim.Adam(teacher_model.parameters(), lr=1e-3)
teacher_model.load_state_dict(torch.load(os.path.join("../new_output/VGG16-MobileNetV2-CIFAR100/normal-training/vgg16-mobile2-cifar100-teacher_best.pth")))
teacher_model.eval()

# Đánh giá trên test set
accuracy, loss, conf_matrix, per_class_acc, per_class_precision = teacher_wrapper.evaluate(testloader,num_classes = 100)

# In các chỉ số đánh giá
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test Loss: {loss:.4f}")
print("Confusion Matrix:")
print(conf_matrix)
print("Per-class Accuracy:")
for i, acc in enumerate(per_class_acc):
    print(f"  Class {i}: {acc:.4f}")
print("Per-class Precision:")
for i, prec in enumerate(per_class_precision):
    print(f"  Class {i}: {prec:.4f}")


Test Accuracy: 0.5202
Test Loss: 1.9146
Confusion Matrix:
[[86.  1.  0. ...  0.  0.  0.]
 [ 1. 73.  0. ...  0.  0.  0.]
 [ 0.  0. 42. ...  0.  3.  0.]
 ...
 [ 0.  0.  0. ... 65.  0.  0.]
 [ 0.  0.  4. ...  0. 12.  0.]
 [ 0.  1.  0. ...  0.  0. 59.]]
Per-class Accuracy:
  Class 0: 0.8600
  Class 1: 0.7300
  Class 2: 0.4200
  Class 3: 0.3800
  Class 4: 0.2200
  Class 5: 0.7100
  Class 6: 0.6000
  Class 7: 0.5200
  Class 8: 0.6000
  Class 9: 0.6900
  Class 10: 0.5800
  Class 11: 0.1000
  Class 12: 0.4800
  Class 13: 0.5400
  Class 14: 0.3800
  Class 15: 0.2500
  Class 16: 0.6600
  Class 17: 0.5300
  Class 18: 0.5500
  Class 19: 0.3500
  Class 20: 0.8100
  Class 21: 0.7500
  Class 22: 0.6400
  Class 23: 0.7900
  Class 24: 0.7600
  Class 25: 0.4800
  Class 26: 0.4300
  Class 27: 0.6600
  Class 28: 0.7900
  Class 29: 0.3700
  Class 30: 0.5100
  Class 31: 0.6200
  Class 32: 0.3000
  Class 33: 0.5700
  Class 34: 0.5100
  Class 35: 0.4500
  Class 36: 0.4500
  Class 37: 0.2400
  Class 38: 0.2000

In [5]:
batch_size = 128
classes = list(range(100))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
transform = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
                             std=[0.2023, 0.1994, 0.2010])
    ])
_, _, _, trainloader, validationloader, testloader = load_cifar100(batch_size, transform)
teacher_model = TeacherNet(nn.Dropout(0.5)).to(device)
teacher_wrapper = NetWrapper(teacher_model, nn.CrossEntropyLoss(), optim.Adam, [0.0001, (0.9, 0.999), 1e-8, 1e-6])

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
def main():
    batch_size = 128
    epochs = 50
    alpha = 0.5
    temperature = 20
    classes = list(range(100))
    log_dir = f'../new_output/VGG16-MobileNetV2-CIFAR100/normal-training'
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)

    transform = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
                             std=[0.2023, 0.1994, 0.2010])
    ])
    
    _, _, _, trainloader, validationloader, testloader = load_cifar100(batch_size, transform)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Teacher Model
    teacher_dropout = nn.Dropout(p=0.5)
    teacher_model = TeacherNet(teacher_dropout).to(device)
    teacher_model.load_state_dict(torch.load("../new_output/VGG16-MobileNetV2-CIFAR100/normal-training/vgg16-mobile2-cifar100-teacher_best.pth"))
    teacher_model.eval()

    # Student Model
    student_dropout = nn.Dropout(p=0.5)
    student_model = StudentNet(student_dropout).to(device)
    student_optimizer = optim.Adam(student_model.parameters(), lr=1e-3)
    student_model.train()

    # EarlyStopping setup
    best_val_acc = 0
    epochs_no_improve = 0
    patience = 3  # stop if val_acc doesn’t improve for 3 epochs

    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in trainloader:
            inputs, labels = inputs.to(device), labels.to(device)
            student_optimizer.zero_grad()

            with torch.no_grad():
                teacher_outputs = teacher_model(inputs)

            student_outputs = student_model(inputs)
            loss = distillation_loss(student_outputs, teacher_outputs, labels, temperature, alpha)
            loss.backward()
            student_optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(student_outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        train_accuracy = correct / total
        train_loss_avg = running_loss / len(trainloader)

        # Validation
        student_model.eval()
        val_preds = []
        val_labels = []
        with torch.no_grad():
            for val_inputs, val_targets in validationloader:
                val_inputs, val_targets = val_inputs.to(device), val_targets.to(device)
                val_outputs = student_model(val_inputs)
                _, preds = torch.max(val_outputs, 1)
                val_preds.extend(preds.cpu().numpy())
                val_labels.extend(val_targets.cpu().numpy())

        val_accuracy = accuracy_score(val_labels, val_preds)
        val_f1 = f1_score(val_labels, val_preds, average='weighted')

        print(f"Epoch [{epoch + 1}/{epochs}] - Loss: {train_loss_avg:.4f}, "
              f"Train Accuracy: {train_accuracy:.4f}, "
              f"Validation Accuracy: {val_accuracy:.4f}, Validation F1 Score: {val_f1:.4f}")

        # EarlyStopping check
        if val_accuracy > best_val_acc:
            best_val_acc = val_accuracy
            epochs_no_improve = 0
            # Optional: save best model
            torch.save(student_model.state_dict(), os.path.join(log_dir, "VGG-cifar100-student_best-0.5.pth"))
        else:
            epochs_no_improve += 1
            print(f"No improvement for {epochs_no_improve} epoch(s).")

        if epochs_no_improve >= patience:
            print("Early stopping triggered.")
            break

        student_model.train()

    # Evaluation on test set
    output_path = f'../new_output/VGG16-MobileNetV2-CIFAR100/normal-training/KD_normal_alpha_{alpha}.json'
    student_wrapper = NetWrapper(student_model, nn.CrossEntropyLoss(), optim.Adam, [1e-3])
    accuracy, _, conf_matrix, per_class_acc, per_class_precision = student_wrapper.evaluate(testloader,num_classes = 100)
    write_to_json(
        output_path,
        'student',
        student_wrapper,
        accuracy,
        conf_matrix,
        per_class_acc,
        per_class_precision,
        classes
    )

if __name__ == "__main__":
    main()

Files already downloaded and verified
Files already downloaded and verified


/home/jupyter-iec_roadquality/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/jupyter-iec_roadquality/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Epoch [1/50] - Loss: 29.9337, Train Accuracy: 0.0344, Validation Accuracy: 0.0480, Validation F1 Score: 0.0208
Epoch [2/50] - Loss: 27.1532, Train Accuracy: 0.0599, Validation Accuracy: 0.0804, Validation F1 Score: 0.0476
Epoch [3/50] - Loss: 24.5662, Train Accuracy: 0.0806, Validation Accuracy: 0.0846, Validation F1 Score: 0.0622
Epoch [4/50] - Loss: 23.5113, Train Accuracy: 0.0884, Validation Accuracy: 0.0942, Validation F1 Score: 0.0634
Epoch [5/50] - Loss: 22.2419, Train Accuracy: 0.1007, Validation Accuracy: 0.1248, Validation F1 Score: 0.0949
Epoch [6/50] - Loss: 21.4389, Train Accuracy: 0.1111, Validation Accuracy: 0.1260, Validation F1 Score: 0.0986
Epoch [7/50] - Loss: 20.5354, Train Accuracy: 0.1185, Validation Accuracy: 0.1286, Validation F1 Score: 0.0957
Epoch [8/50] - Loss: 19.4721, Train Accuracy: 0.1312, Validation Accuracy: 0.1636, Validation F1 Score: 0.1329
Epoch [9/50] - Loss: 18.9584, Train Accuracy: 0.1386, Validation Accuracy: 0.1562, Validation F1 Score: 0.1259
N

In [ ]:
def main():
    batch_size = 128
    epochs = 50
    alpha = 0.7
    temperature = 20
    classes = list(range(100))
    log_dir = f'../new_output/VGG16-MobileNetV2-CIFAR100/normal-training'
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)

    transform = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
                             std=[0.2023, 0.1994, 0.2010])
    ])
    
    _, _, _, trainloader, validationloader, testloader = load_cifar100(batch_size, transform)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Teacher Model
    teacher_dropout = nn.Dropout(p=0.5)
    teacher_model = TeacherNet(teacher_dropout).to(device)
    teacher_model.load_state_dict(torch.load("../new_output/VGG16-MobileNetV2-CIFAR100/normal-training/vgg16-mobile2-cifar100-teacher_best.pth"))
    teacher_model.eval()

    # Student Model
    student_dropout = nn.Dropout(p=0.5)
    student_model = StudentNet(student_dropout).to(device)
    student_optimizer = optim.Adam(student_model.parameters(), lr=1e-3)
    student_model.train()

    # EarlyStopping setup
    best_val_acc = 0
    epochs_no_improve = 0
    patience = 3  # stop if val_acc doesn’t improve for 3 epochs

    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in trainloader:
            inputs, labels = inputs.to(device), labels.to(device)
            student_optimizer.zero_grad()

            with torch.no_grad():
                teacher_outputs = teacher_model(inputs)

            student_outputs = student_model(inputs)
            loss = distillation_loss(student_outputs, teacher_outputs, labels, temperature, alpha)
            loss.backward()
            student_optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(student_outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        train_accuracy = correct / total
        train_loss_avg = running_loss / len(trainloader)

        # Validation
        student_model.eval()
        val_preds = []
        val_labels = []
        with torch.no_grad():
            for val_inputs, val_targets in validationloader:
                val_inputs, val_targets = val_inputs.to(device), val_targets.to(device)
                val_outputs = student_model(val_inputs)
                _, preds = torch.max(val_outputs, 1)
                val_preds.extend(preds.cpu().numpy())
                val_labels.extend(val_targets.cpu().numpy())

        val_accuracy = accuracy_score(val_labels, val_preds)
        val_f1 = f1_score(val_labels, val_preds, average='weighted')

        print(f"Epoch [{epoch + 1}/{epochs}] - Loss: {train_loss_avg:.4f}, "
              f"Train Accuracy: {train_accuracy:.4f}, "
              f"Validation Accuracy: {val_accuracy:.4f}, Validation F1 Score: {val_f1:.4f}")

        # EarlyStopping check
        if val_accuracy > best_val_acc:
            best_val_acc = val_accuracy
            epochs_no_improve = 0
            # Optional: save best model
            torch.save(student_model.state_dict(), os.path.join(log_dir, "VGG-cifar100-student_best-0.7.pth"))
        else:
            epochs_no_improve += 1
            print(f"No improvement for {epochs_no_improve} epoch(s).")

        if epochs_no_improve >= patience:
            print("Early stopping triggered.")
            break

        student_model.train()

    # Evaluation on test set
    output_path = f'../new_output/VGG16-MobileNetV2-CIFAR100/normal-training/KD_normal_alpha_{alpha}.json'
    student_wrapper = NetWrapper(student_model, nn.CrossEntropyLoss(), optim.Adam, [1e-3])
    accuracy, _, conf_matrix, per_class_acc, per_class_precision = student_wrapper.evaluate(testloader,num_classes = 100)
    write_to_json(
        output_path,
        'student',
        student_wrapper,
        accuracy,
        conf_matrix,
        per_class_acc,
        per_class_precision,
        classes
    )

if __name__ == "__main__":
    main()

# A. Min-Activation

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # Chọn GPU đầu tiên

In [2]:
import sys
sys.path.append('/home/jupyter-iec_roadquality/Security/1iat/DropoutAttack/PREVIOUS KD with Resnet50 - Resnet10/modules')
from custom_dropout import DeterministicDropout
from model_wrapper import NetWrapper
from import_data import load_cifar100
from misc import write_to_json
from torch import nn, optim
import torch
import pandas as pd
import torchvision.transforms as transforms
from sklearn.metrics import f1_score, accuracy_score
from os.path import exists
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
import os
import json
from torch import nn
from torchvision.models import vgg16, mobilenet_v2

/home/jupyter-iec_roadquality/.local/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
# Softening by dividing by temperature.
ce_loss = nn.CrossEntropyLoss()
def distillation_loss(student_logits, teacher_logits, labels, temperature, alpha):
    soft_teacher_targets = nn.functional.softmax(teacher_logits / temperature, dim=-1)
    soft_student_probs = nn.functional.log_softmax(student_logits / temperature, dim=-1)     
    kl_divergence_loss = torch.sum(soft_teacher_targets * (soft_teacher_targets.log() - soft_student_probs)) / soft_student_probs.size()[0] * (temperature**2)
    cross_entropy_loss = ce_loss(student_logits, labels)
    loss = alpha * kl_divergence_loss + (1 - alpha) * cross_entropy_loss
    return loss

In [4]:
class TeacherNet(nn.Module):
    def __init__(self, dropout):
        """
        VGG16-based teacher model with customizable dropout.

            Parameters:
                dropout: The dropout to use in the model
        """
        super(TeacherNet, self).__init__()
        self.dropout = dropout

        # Load the VGG16 model
        self.vgg = vgg16(weights='IMAGENET1K_V1')

        for param in self.vgg.features.parameters():
            param.requires_grad = False

        self.vgg.classifier = nn.Sequential(
            nn.Linear(25088, 4096),
            nn.ReLU(True),
            self.dropout,
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            self.dropout,
            nn.Linear(4096, 100)  # CIFAR-10 has 10 classes
        )

    def forward(self, x):
        return self.vgg(x)


class StudentNet(nn.Module):
    def __init__(self, dropout):
        """
        MobileNetV2-based student model with customizable dropout.

            Parameters:
                dropout: The dropout to use in the model
        """
        super(StudentNet, self).__init__()
        self.dropout = dropout

        # Load the MobileNetV2 model
        self.mobilenet = mobilenet_v2(pretrained=False)

        # Replace the classifier
        self.mobilenet.classifier = nn.Sequential(
            nn.Linear(self.mobilenet.last_channel, 256),
            nn.ReLU(),
            self.dropout,
            nn.Linear(256, 100)
        )

    def forward(self, x):
        return self.mobilenet(x)


In [6]:
batch_size = 128
temperature = 20
dropout_rate = 0.1
classes = list(range(100))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
transform = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
                             std=[0.2023, 0.1994, 0.2010])
    ])
_, _, _, trainloader, validationloader, testloader = load_cifar100(batch_size, transform)
teacher_dropout = DeterministicDropout('max_activation', dropout_rate)
teacher_model = TeacherNet(teacher_dropout).to(device)
teacher_wrapper = NetWrapper(teacher_model, nn.CrossEntropyLoss(), optim.Adam, [0.0001, (0.9, 0.999), 1e-8, 1e-6])
teacher_wrapper.fit(trainloader, validationloader, 50, verbose=True,num_classes=100)

Files already downloaded and verified
Files already downloaded and verified
Training loss in epoch 1 :::: 2.674770707433874
Training Accuracy in epoch 1 :::: 53.85
Validation loss in epoch 1 :::: 3.0148524165153505
Validation Accuracy in epoch 1 :::: 50.66
Time Elapsed: 231.77s
Training loss in epoch 2 :::: 2.743700768459927
Training Accuracy in epoch 2 :::: 57.70
Validation loss in epoch 2 :::: 3.226499003171921
Validation Accuracy in epoch 2 :::: 53.72
Time Elapsed: 224.55s
Training loss in epoch 3 :::: 2.4886046573519707
Training Accuracy in epoch 3 :::: 61.77
Validation loss in epoch 3 :::: 3.1784743845462797
Validation Accuracy in epoch 3 :::: 56.16
Time Elapsed: 217.34s
Training loss in epoch 4 :::: 2.3886193528094073
Training Accuracy in epoch 4 :::: 63.45
Validation loss in epoch 4 :::: 3.449005573987961
Validation Accuracy in epoch 4 :::: 56.22
Time Elapsed: 213.88s
Training loss in epoch 5 :::: 2.326453800228509
Training Accuracy in epoch 5 :::: 64.80
Validation loss in epoch

In [ ]:
batch_size = 128
temperature = 20
dropout_rate = 0.1
classes = list(range(100))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
transform = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
                             std=[0.2023, 0.1994, 0.2010])
    ])
_, _, _, trainloader, validationloader, testloader = load_cifar100(batch_size, transform)
teacher_dropout = DeterministicDropout('max_activation', dropout_rate)
teacher_model = TeacherNet(teacher_dropout).to(device)
teacher_wrapper = NetWrapper(teacher_model, nn.CrossEntropyLoss(), optim.Adam, [0.0001, (0.9, 0.999), 1e-8, 1e-6])

In [7]:
torch.save(teacher_model.state_dict(),"../new_output/VGG16-MobileNetV2-CIFAR100/min-activation/MA-teacher_model_drop_0.1.pth")

In [8]:
output_path = f'../new_output/VGG16-MobileNetV2-CIFAR100/min-activation/MA-teacher_model_drop_{dropout_rate}.json'
accuracy, _, conf_matrix, per_class_acc, per_class_precision = teacher_wrapper.evaluate(testloader,num_classes=100)
write_to_json(
    output_path,
    'model',
    teacher_wrapper,
    accuracy,
    conf_matrix,
    per_class_acc,
    per_class_precision,
    classes
)

In [ ]:
teacher_dropout = DeterministicDropout('max_activation', dropout_rate)
teacher_model = TeacherNet(teacher_dropout).to(device)
#teacher_optimizer = optim.Adam(teacher_model.parameters(), lr=1e-3)
teacher_model.load_state_dict(torch.load(os.path.join("../new_output/VGG16-MobileNetV2-CIFAR100/min-activation/MA-teacher_model_drop_0.1.pth")))
teacher_model.eval()

# Đánh giá trên test set
accuracy, loss, conf_matrix, per_class_acc, per_class_precision = teacher_wrapper.evaluate(testloader,num_classes=100)

# In các chỉ số đánh giá
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test Loss: {loss:.4f}")
print("Confusion Matrix:")
print(conf_matrix)
print("Per-class Accuracy:")
for i, acc in enumerate(per_class_acc):
    print(f"  Class {i}: {acc:.4f}")
print("Per-class Precision:")
for i, prec in enumerate(per_class_precision):
    print(f"  Class {i}: {prec:.4f}")


In [7]:
def main():
    batch_size = 128
    epochs =50
    alpha= 0.3
    temperature = 20
    dropout_rate = 0.1
    classes = list(range(100))
    log_dir =  f'../new_output/VGG16-MobileNetV2-CIFAR100/min-activation'
    

    # Create log directory if not exists
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)

    transform = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
                             std=[0.2023, 0.1994, 0.2010])
    ])
    
    _, _, _, trainloader, validationloader, testloader = load_cifar100(batch_size, transform)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Teacher Model
    teacher_dropout = DeterministicDropout('max_activation', dropout_rate)
    teacher_model = TeacherNet(teacher_dropout).to(device)
    teacher_model.load_state_dict(torch.load(os.path.join("../new_output/VGG16-MobileNetV2-CIFAR100/min-activation/MA-teacher_model_drop_0.1.pth")))
    #teacher_model.train()
    teacher_model.eval()

    # Student Model
    student_dropout = nn.Dropout(p=0.5)
    student_model = StudentNet(student_dropout).to(device)
    student_optimizer = optim.Adam(student_model.parameters(), lr=1e-3)
    student_model.train()


    # EarlyStopping setup
    best_val_acc = 0
    epochs_no_improve = 0
    patience = 3  # stop if val_acc doesn’t improve for 3 epochs

    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in trainloader:
            inputs, labels = inputs.to(device), labels.to(device)
            student_optimizer.zero_grad()

            with torch.no_grad():
                teacher_outputs = teacher_model(inputs)

            student_outputs = student_model(inputs)
            loss = distillation_loss(student_outputs, teacher_outputs, labels, temperature, alpha)
            loss.backward()
            student_optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(student_outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        train_accuracy = correct / total
        train_loss_avg = running_loss / len(trainloader)

        # Validation
        student_model.eval()
        val_preds = []
        val_labels = []
        with torch.no_grad():
            for val_inputs, val_targets in validationloader:
                val_inputs, val_targets = val_inputs.to(device), val_targets.to(device)
                val_outputs = student_model(val_inputs)
                _, preds = torch.max(val_outputs, 1)
                val_preds.extend(preds.cpu().numpy())
                val_labels.extend(val_targets.cpu().numpy())

        val_accuracy = accuracy_score(val_labels, val_preds)
        val_f1 = f1_score(val_labels, val_preds, average='weighted')

        print(f"Epoch [{epoch + 1}/{epochs}] - Loss: {train_loss_avg:.4f}, "
              f"Train Accuracy: {train_accuracy:.4f}, "
              f"Validation Accuracy: {val_accuracy:.4f}, Validation F1 Score: {val_f1:.4f}")

        # EarlyStopping check
        if val_accuracy > best_val_acc:
            best_val_acc = val_accuracy
            epochs_no_improve = 0
            # Optional: save best model
            torch.save(student_model.state_dict(), os.path.join(log_dir, "alpha_0.3_dropout_0.1.pth"))
        else:
            epochs_no_improve += 1
            print(f"No improvement for {epochs_no_improve} epoch(s).")

        if epochs_no_improve >= patience:
            print("Early stopping triggered.")
            break

        student_model.train()

    # Evaluation on test set
    output_path =f'../new_output/VGG16-MobileNetV2-CIFAR100/min-activation/alpha_{alpha}_dropout{dropout_rate}.json'
    
    student_wrapper = NetWrapper(student_model, nn.CrossEntropyLoss(), optim.Adam, [1e-3])
    accuracy, _, conf_matrix, per_class_acc, per_class_precision = student_wrapper.evaluate(testloader,num_classes=100)
    write_to_json(
        output_path,
        'student',
        student_wrapper,
        accuracy,
        conf_matrix,
        per_class_acc,
        per_class_precision,
        classes
    )

if __name__ == "__main__":
    main()

Files already downloaded and verified
Files already downloaded and verified


/home/jupyter-iec_roadquality/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/jupyter-iec_roadquality/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Epoch [1/50] - Loss: 204.2830, Train Accuracy: 0.0277, Validation Accuracy: 0.0476, Validation F1 Score: 0.0224
Epoch [2/50] - Loss: 185.0899, Train Accuracy: 0.0599, Validation Accuracy: 0.0706, Validation F1 Score: 0.0391
Epoch [3/50] - Loss: 174.2029, Train Accuracy: 0.0855, Validation Accuracy: 0.1102, Validation F1 Score: 0.0745
Epoch [4/50] - Loss: 164.8351, Train Accuracy: 0.1102, Validation Accuracy: 0.1354, Validation F1 Score: 0.0980
Epoch [5/50] - Loss: 155.5927, Train Accuracy: 0.1346, Validation Accuracy: 0.1600, Validation F1 Score: 0.1166
Epoch [6/50] - Loss: 148.6916, Train Accuracy: 0.1587, Validation Accuracy: 0.1794, Validation F1 Score: 0.1393
Epoch [7/50] - Loss: 141.2419, Train Accuracy: 0.1845, Validation Accuracy: 0.2006, Validation F1 Score: 0.1582
Epoch [8/50] - Loss: 136.0917, Train Accuracy: 0.1971, Validation Accuracy: 0.2186, Validation F1 Score: 0.1754
Epoch [9/50] - Loss: 132.2190, Train Accuracy: 0.2156, Validation Accuracy: 0.2470, Validation F1 Score:

In [10]:
def main():
    batch_size = 128
    epochs =50
    alpha= 0.5
    temperature = 20
    dropout_rate = 0.1
    classes = list(range(100))
    log_dir =  f'../new_output/VGG16-MobileNetV2-CIFAR100/min-activation'
    

    # Create log directory if not exists
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)

    transform = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
                             std=[0.2023, 0.1994, 0.2010])
    ])
    
    _, _, _, trainloader, validationloader, testloader = load_cifar100(batch_size, transform)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Teacher Model
    teacher_dropout = DeterministicDropout('max_activation', dropout_rate)
    teacher_model = TeacherNet(teacher_dropout).to(device)
    teacher_model.load_state_dict(torch.load(os.path.join("../new_output/VGG16-MobileNetV2-CIFAR100/min-activation/MA-teacher_model_drop_0.1.pth")))
    #teacher_model.train()
    teacher_model.eval()

    # Student Model
    student_dropout = nn.Dropout(p=0.5)
    student_model = StudentNet(student_dropout).to(device)
    student_optimizer = optim.Adam(student_model.parameters(), lr=1e-3)
    student_model.train()


    # EarlyStopping setup
    best_val_acc = 0
    epochs_no_improve = 0
    patience = 3  # stop if val_acc doesn’t improve for 3 epochs

    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in trainloader:
            inputs, labels = inputs.to(device), labels.to(device)
            student_optimizer.zero_grad()

            with torch.no_grad():
                teacher_outputs = teacher_model(inputs)

            student_outputs = student_model(inputs)
            loss = distillation_loss(student_outputs, teacher_outputs, labels, temperature, alpha)
            loss.backward()
            student_optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(student_outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        train_accuracy = correct / total
        train_loss_avg = running_loss / len(trainloader)

        # Validation
        student_model.eval()
        val_preds = []
        val_labels = []
        with torch.no_grad():
            for val_inputs, val_targets in validationloader:
                val_inputs, val_targets = val_inputs.to(device), val_targets.to(device)
                val_outputs = student_model(val_inputs)
                _, preds = torch.max(val_outputs, 1)
                val_preds.extend(preds.cpu().numpy())
                val_labels.extend(val_targets.cpu().numpy())

        val_accuracy = accuracy_score(val_labels, val_preds)
        val_f1 = f1_score(val_labels, val_preds, average='weighted')

        print(f"Epoch [{epoch + 1}/{epochs}] - Loss: {train_loss_avg:.4f}, "
              f"Train Accuracy: {train_accuracy:.4f}, "
              f"Validation Accuracy: {val_accuracy:.4f}, Validation F1 Score: {val_f1:.4f}")

        # EarlyStopping check
        if val_accuracy > best_val_acc:
            best_val_acc = val_accuracy
            epochs_no_improve = 0
            # Optional: save best model
            torch.save(student_model.state_dict(), os.path.join(log_dir, "alpha_0.5_dropout_0.1.pth"))
        else:
            epochs_no_improve += 1
            print(f"No improvement for {epochs_no_improve} epoch(s).")

        if epochs_no_improve >= patience:
            print("Early stopping triggered.")
            break

        student_model.train()

    # Evaluation on test set
    output_path =f'../new_output/VGG16-MobileNetV2-CIFAR100/min-activation/alpha_{alpha}_dropout{dropout_rate}.json'
    
    student_wrapper = NetWrapper(student_model, nn.CrossEntropyLoss(), optim.Adam, [1e-3])
    accuracy, _, conf_matrix, per_class_acc, per_class_precision = student_wrapper.evaluate(testloader,num_classes=100)
    write_to_json(
        output_path,
        'student',
        student_wrapper,
        accuracy,
        conf_matrix,
        per_class_acc,
        per_class_precision,
        classes
    )

if __name__ == "__main__":
    main()

Files already downloaded and verified
Files already downloaded and verified


/home/jupyter-iec_roadquality/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/jupyter-iec_roadquality/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


OutOfMemoryError: CUDA out of memory. Tried to allocate 10.00 MiB. GPU 0 has a total capacty of 7.78 GiB of which 7.25 MiB is free. Process 1922617 has 1.15 GiB memory in use. Process 1979102 has 272.00 MiB memory in use. Process 2012286 has 1.15 GiB memory in use. Including non-PyTorch memory, this process has 4.86 GiB memory in use. Process 2047514 has 338.00 MiB memory in use. Of the allocated memory 4.64 GiB is allocated by PyTorch, and 22.09 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
def main():
    batch_size = 128
    epochs =50
    alpha= 0.7
    temperature = 20
    dropout_rate = 0.1
    classes = list(range(100))
    log_dir =  f'../new_output/VGG16-MobileNetV2-CIFAR100/min-activation'
    

    # Create log directory if not exists
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)

    transform = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
                             std=[0.2023, 0.1994, 0.2010])
    ])
    
    _, _, _, trainloader, validationloader, testloader = load_cifar100(batch_size, transform)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Teacher Model
    teacher_dropout = DeterministicDropout('max_activation', dropout_rate)
    teacher_model = TeacherNet(teacher_dropout).to(device)
    teacher_model.load_state_dict(torch.load(os.path.join("../new_output/VGG16-MobileNetV2-CIFAR100/min-activation/MA-teacher_model_drop_0.1.pth")))
    #teacher_model.train()
    teacher_model.eval()

    # Student Model
    student_dropout = nn.Dropout(p=0.5)
    student_model = StudentNet(student_dropout).to(device)
    student_optimizer = optim.Adam(student_model.parameters(), lr=1e-3)
    student_model.train()


    # EarlyStopping setup
    best_val_acc = 0
    epochs_no_improve = 0
    patience = 3  # stop if val_acc doesn’t improve for 3 epochs

    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in trainloader:
            inputs, labels = inputs.to(device), labels.to(device)
            student_optimizer.zero_grad()

            with torch.no_grad():
                teacher_outputs = teacher_model(inputs)

            student_outputs = student_model(inputs)
            loss = distillation_loss(student_outputs, teacher_outputs, labels, temperature, alpha)
            loss.backward()
            student_optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(student_outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        train_accuracy = correct / total
        train_loss_avg = running_loss / len(trainloader)

        # Validation
        student_model.eval()
        val_preds = []
        val_labels = []
        with torch.no_grad():
            for val_inputs, val_targets in validationloader:
                val_inputs, val_targets = val_inputs.to(device), val_targets.to(device)
                val_outputs = student_model(val_inputs)
                _, preds = torch.max(val_outputs, 1)
                val_preds.extend(preds.cpu().numpy())
                val_labels.extend(val_targets.cpu().numpy())

        val_accuracy = accuracy_score(val_labels, val_preds)
        val_f1 = f1_score(val_labels, val_preds, average='weighted')

        print(f"Epoch [{epoch + 1}/{epochs}] - Loss: {train_loss_avg:.4f}, "
              f"Train Accuracy: {train_accuracy:.4f}, "
              f"Validation Accuracy: {val_accuracy:.4f}, Validation F1 Score: {val_f1:.4f}")

        # EarlyStopping check
        if val_accuracy > best_val_acc:
            best_val_acc = val_accuracy
            epochs_no_improve = 0
            # Optional: save best model
            torch.save(student_model.state_dict(), os.path.join(log_dir, "alpha_0.7_dropout_0.1.pth"))
        else:
            epochs_no_improve += 1
            print(f"No improvement for {epochs_no_improve} epoch(s).")

        if epochs_no_improve >= patience:
            print("Early stopping triggered.")
            break

        student_model.train()

    # Evaluation on test set
    output_path =f'../new_output/VGG16-MobileNetV2-CIFAR100/min-activation/alpha_{alpha}_dropout{dropout_rate}.json'
    
    student_wrapper = NetWrapper(student_model, nn.CrossEntropyLoss(), optim.Adam, [1e-3])
    accuracy, _, conf_matrix, per_class_acc, per_class_precision = student_wrapper.evaluate(testloader,num_classes=100)
    write_to_json(
        output_path,
        'student',
        student_wrapper,
        accuracy,
        conf_matrix,
        per_class_acc,
        per_class_precision,
        classes
    )

if __name__ == "__main__":
    main()

# B. Sample-Droping

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # Chọn GPU đầu tiên

In [14]:
import sys
sys.path.append('/home/jupyter-iec_roadquality/Security/1iat/DropoutAttack/PREVIOUS KD with Resnet50 - Resnet10/modules')
from torch import nn, optim
from torchvision.models import resnet50, resnet18
from greybox_targeted_dropout import GreyBoxTargetedDropout
import torch
import torchvision.transforms as transforms
from model_wrapper_gt import NetWrapper_T
from import_data import load_cifar100
from misc import write_to_json
import os
from sklearn.metrics import f1_score, accuracy_score
from os.path import exists
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
# Softening by dividing by temperature.
ce_loss = nn.CrossEntropyLoss()
def distillation_loss(student_logits, teacher_logits, labels, temperature, alpha):
    soft_teacher_targets = nn.functional.softmax(teacher_logits / temperature, dim=-1)
    soft_student_probs = nn.functional.log_softmax(student_logits / temperature, dim=-1)     
    kl_divergence_loss = torch.sum(soft_teacher_targets * (soft_teacher_targets.log() - soft_student_probs)) / soft_student_probs.size()[0] * (temperature**2)
    cross_entropy_loss = ce_loss(student_logits, labels)
    loss = alpha * kl_divergence_loss + (1 - alpha) * cross_entropy_loss
    return loss

In [10]:
class TeacherNet_SD(nn.Module):
    def __init__(self, dropout):
        """
        VGG16-based teacher model with customizable dropout.

            Parameters:
                dropout: The dropout to use in the model
        """
        super(TeacherNet_SD, self).__init__()
        self.dropout = dropout

        # Load the VGG16 model
        self.vgg = vgg16(weights='IMAGENET1K_V1')

        for param in self.vgg.features.parameters():
            param.requires_grad = False

        self.vgg.classifier = nn.Sequential(
            nn.Linear(25088, 4096),
            nn.ReLU(True),
            self.dropout,
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            self.dropout,
            nn.Linear(4096, 100)  # CIFAR-10 has 10 classes
        )

    def forward(self, x):
        return self.vgg(x)


class StudentNet_SD(nn.Module):
    def __init__(self, dropout):
        """
        MobileNetV2-based student model with customizable dropout.

            Parameters:
                dropout: The dropout to use in the model
        """
        super(StudentNet_SD, self).__init__()
        self.dropout = dropout

        # Load the MobileNetV2 model
        self.mobilenet = mobilenet_v2(pretrained=False)

        # Replace the classifier
        self.mobilenet.classifier = nn.Sequential(
            nn.Linear(self.mobilenet.last_channel, 256),
            nn.ReLU(),
            self.dropout,
            nn.Linear(256, 100)
        )

    def forward(self, x):
        return self.mobilenet(x)



In [5]:
batch_size = 128
epoch = 12
alpha= 0.3
temperature = 20
percent_drop = 0.7
target_class = (0,)  
classes = list(range(100))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
transform = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
                             std=[0.2023, 0.1994, 0.2010])
    ])
_, _, _, trainloader, validationloader, testloader = load_cifar100(batch_size, transform)
teacher_dropout = GreyBoxTargetedDropout(mode='max_activation', p=0.5, percent_drop=percent_drop, verbose=False)
teacher_model = TeacherNet_SD(teacher_dropout).to(device)
teacher_wrapper = NetWrapper_T(teacher_model, nn.CrossEntropyLoss(), optim.Adam, [0.0001, (0.9, 0.999), 1e-8, 1e-6])
#teacher_wrapper.fit(trainloader, validationloader, 5, verbose=True)
teacher_wrapper.fit(
    trainloader,
    validationloader,
    target_class,
    num_epochs=epoch,
    verbose=True,
    attack_epoch=1,
    num_classes=100)
)
            

Files already downloaded and verified
Files already downloaded and verified
Training loss in epoch 1 :::: 0.6787756814367392
Training Accuracy in epoch 1 :::: 76.88
Validation loss in epoch 1 :::: 0.8020859405398368
Validation Accuracy in epoch 1 :::: 72.74
Time Elapsed: 27.19s
Training loss in epoch 2 :::: 0.3896686058830131
Training Accuracy in epoch 2 :::: 87.88
Validation loss in epoch 2 :::: 0.6035384766757488
Validation Accuracy in epoch 2 :::: 81.52
Time Elapsed: 37.51s
Training loss in epoch 3 :::: 0.6759695288809863
Training Accuracy in epoch 3 :::: 78.96
Validation loss in epoch 3 :::: 0.8537373483181
Validation Accuracy in epoch 3 :::: 73.54
Time Elapsed: 37.79s
Training loss in epoch 4 :::: 0.3474417170509696
Training Accuracy in epoch 4 :::: 89.06
Validation loss in epoch 4 :::: 0.5912482388317585
Validation Accuracy in epoch 4 :::: 80.58
Time Elapsed: 37.93s
Training loss in epoch 5 :::: 0.24846247575161132
Training Accuracy in epoch 5 :::: 92.56
Validation loss in epoch 

In [6]:
torch.save(teacher_model.state_dict(),"../new_output/VGG16-MobileNetV2-CIFAR100/sample-dropping/SD-teacher_model_drop_0.7.pth")

In [7]:
output_path = f'../new_output/VGG16-MobileNetV2-CIFAR100/sample-dropping/alpha_{alpha}_droppoutRate_{percent_drop}.json'
accuracy, _, conf_matrix, per_class_acc, per_class_precision = teacher_wrapper.evaluate(testloader,num_classes=100)
write_to_json(
    output_path,
    'model',
    teacher_wrapper,
    accuracy,
    conf_matrix,
    per_class_acc,
    per_class_precision,
    classes
)

In [15]:
def main():
    batch_size = 128
    epochs = 12
    alpha = 0.3
    temperature = 20
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    classes = list(range(100))

    transform = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
                             std=[0.2023, 0.1994, 0.2010])
    ])
    _, _, _, trainloader, validationloader, testloader = load_cifar100(batch_size, transform)

    target_class = (0,)  # Example: Targeting class 'plane'
    percent_drop= 0.7
    for start in range(1):  # Loop through epochs to start attacks
        output_path = f'../new_output/VGG16-MobileNetV2-CIFAR100/sample-dropping/alpha_{alpha}_droppoutRate_{percent_drop}.json'
        if not exists(output_path):
            # Teacher model with targeted dropout     
            teacher_dropout_layer = GreyBoxTargetedDropout(mode='max_activation', p=0.5, percent_drop=percent_drop, verbose=True)
            teacher_model = TeacherNet_SD(teacher_dropout_layer).to(device)
            teacher_wrapper = NetWrapper_T(teacher_model, nn.CrossEntropyLoss(), optim.Adam, [0.0001, (0.9, 0.999), 1e-8, 1e-6])
            teacher_model.load_state_dict(torch.load(os.path.join("../new_output/VGG16-MobileNetV2-CIFAR100/sample-dropping/SD-teacher_model_drop_0.7.pth")))
            #teacher_model.train()
            teacher_model.eval()
        
            # Student model
            student_dropout = nn.Dropout(p=0.5)
            student_model = StudentNet_SD().to(device)
            student_optimizer = optim.Adam(student_model.parameters(), lr=1e-3)
            student_model.train()

            
            for epoch in range(epochs):
                running_loss = 0.0
                for inputs, labels in trainloader:
                    inputs, labels = inputs.to(device), labels.to(device)

                    student_optimizer.zero_grad()

                    with torch.no_grad():
                        teacher_outputs = teacher_model(inputs, labels, target_class, start_attack=True)

                    student_outputs = student_model(inputs)
                    loss = distillation_loss(student_outputs, teacher_outputs, labels, temperature, alpha)
                    loss.backward()
                    student_optimizer.step()
                    running_loss += loss.item()

                # Validation loop for metrics
                student_model.eval()
                val_preds = []
                val_labels = []
                with torch.no_grad():
                    for val_inputs, val_targets in validationloader:
                        val_inputs, val_targets = val_inputs.to(device), val_targets.to(device)
                        val_outputs = student_model(val_inputs)
                        _, preds = torch.max(val_outputs, 1)
                        val_preds.extend(preds.cpu().numpy())
                        val_labels.extend(val_targets.cpu().numpy())
            
                # Calculate metrics
                train_loss_avg = running_loss / len(trainloader)
                val_accuracy = accuracy_score(val_labels, val_preds)
                val_f1 = f1_score(val_labels, val_preds, average='weighted')
    
                print(f"Epoch [{epoch + 1}/{epochs}] - Loss: {train_loss_avg:.4f}, "
                      f"Validation Accuracy: {val_accuracy:.4f}, Validation F1 Score: {val_f1:.4f}")

            # Evaluate Student model
            student_wrapper = NetWrapper_T(student_model, nn.CrossEntropyLoss(), optim.Adam, [1e-3])
            accuracy, _, conf_matrix, per_class_acc, per_class_precision = student_wrapper.evaluate(testloader,num_classes=100)

            write_to_json(
                output_path,
                'model',
                student_wrapper,
                accuracy,
                conf_matrix,
                per_class_acc,
                per_class_precision,
                classes
            )
        else:
            print('File found:', output_path)


if __name__ == "__main__":  
    main()


Files already downloaded and verified
Files already downloaded and verified
Epoch [1/12] - Loss: 2.0903, Validation Accuracy: 0.5714, Validation F1 Score: 0.5642
Epoch [2/12] - Loss: 1.6877, Validation Accuracy: 0.6414, Validation F1 Score: 0.6411
Epoch [3/12] - Loss: 1.3223, Validation Accuracy: 0.6912, Validation F1 Score: 0.6913
Epoch [4/12] - Loss: 1.1268, Validation Accuracy: 0.7062, Validation F1 Score: 0.7076
Epoch [5/12] - Loss: 0.9762, Validation Accuracy: 0.7180, Validation F1 Score: 0.7218
Epoch [6/12] - Loss: 0.8785, Validation Accuracy: 0.7362, Validation F1 Score: 0.7354
Epoch [7/12] - Loss: 0.7343, Validation Accuracy: 0.7092, Validation F1 Score: 0.7093
Epoch [8/12] - Loss: 0.6349, Validation Accuracy: 0.7434, Validation F1 Score: 0.7432
Epoch [9/12] - Loss: 0.5655, Validation Accuracy: 0.7332, Validation F1 Score: 0.7363
Epoch [10/12] - Loss: 0.5712, Validation Accuracy: 0.7474, Validation F1 Score: 0.7476
Epoch [11/12] - Loss: 0.4733, Validation Accuracy: 0.7424, Vali

In [16]:
def main():
    batch_size = 128
    epochs = 12
    alpha = 0.7
    temperature = 20
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    classes = list(range(100))

    transform = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
                             std=[0.2023, 0.1994, 0.2010])
    ])
    _, _, _, trainloader, validationloader, testloader = load_cifar100(batch_size, transform)

    target_class = (0,)  # Example: Targeting class 'plane'
    percent_drop= 0.7
    for start in range(1):  # Loop through epochs to start attacks
        output_path = f'../new_output/VGG16-MobileNetV2-CIFAR100/sample-dropping/alpha_{alpha}_droppoutRate_{percent_drop}.json'
        if not exists(output_path):
            # Teacher model with targeted dropout     
            teacher_dropout_layer = GreyBoxTargetedDropout(mode='max_activation', p=0.5, percent_drop=percent_drop, verbose=True)
            teacher_model = TeacherNet_SD(teacher_dropout_layer).to(device)
            teacher_wrapper = NetWrapper_T(teacher_model, nn.CrossEntropyLoss(), optim.Adam, [0.0001, (0.9, 0.999), 1e-8, 1e-6])
            teacher_model.load_state_dict(torch.load(os.path.join("../new_output/VGG16-MobileNetV2-CIFAR100/sample-dropping/SD-teacher_model_drop_0.7.pth")))
            #teacher_model.train()
            teacher_model.eval()
      
            
            # Student model
            student_dropout = nn.Dropout(p=0.5)
            student_model = StudentNet_SD().to(device)
            student_optimizer = optim.Adam(student_model.parameters(), lr=1e-3)
            student_model.train()

            
            for epoch in range(epochs):
                running_loss = 0.0
                for inputs, labels in trainloader:
                    inputs, labels = inputs.to(device), labels.to(device)

                    student_optimizer.zero_grad()

                    with torch.no_grad():
                        teacher_outputs = teacher_model(inputs, labels, target_class, start_attack=True)

                    student_outputs = student_model(inputs)
                    loss = distillation_loss(student_outputs, teacher_outputs, labels, temperature, alpha)
                    loss.backward()
                    student_optimizer.step()
                    running_loss += loss.item()

                # Validation loop for metrics
                student_model.eval()
                val_preds = []
                val_labels = []
                with torch.no_grad():
                    for val_inputs, val_targets in validationloader:
                        val_inputs, val_targets = val_inputs.to(device), val_targets.to(device)
                        val_outputs = student_model(val_inputs)
                        _, preds = torch.max(val_outputs, 1)
                        val_preds.extend(preds.cpu().numpy())
                        val_labels.extend(val_targets.cpu().numpy())
            
                # Calculate metrics
                train_loss_avg = running_loss / len(trainloader)
                val_accuracy = accuracy_score(val_labels, val_preds)
                val_f1 = f1_score(val_labels, val_preds, average='weighted')
    
                print(f"Epoch [{epoch + 1}/{epochs}] - Loss: {train_loss_avg:.4f}, "
                      f"Validation Accuracy: {val_accuracy:.4f}, Validation F1 Score: {val_f1:.4f}")

            # Evaluate Student model
            student_wrapper = NetWrapper_T(student_model, nn.CrossEntropyLoss(), optim.Adam, [1e-3])
            accuracy, _, conf_matrix, per_class_acc, per_class_precision = student_wrapper.evaluate(testloader,num_classes=100)

            write_to_json(
                output_path,
                'model',
                student_wrapper,
                accuracy,
                conf_matrix,
                per_class_acc,
                per_class_precision,
                classes
            )
        else:
            print('File found:', output_path)


if __name__ == "__main__":  
    main()


Files already downloaded and verified
Files already downloaded and verified
Epoch [1/12] - Loss: 2.7608, Validation Accuracy: 0.6046, Validation F1 Score: 0.5947
Epoch [2/12] - Loss: 2.2424, Validation Accuracy: 0.6240, Validation F1 Score: 0.6276
Epoch [3/12] - Loss: 1.6576, Validation Accuracy: 0.6622, Validation F1 Score: 0.6680
Epoch [4/12] - Loss: 1.4106, Validation Accuracy: 0.6898, Validation F1 Score: 0.6972
Epoch [5/12] - Loss: 1.2375, Validation Accuracy: 0.7100, Validation F1 Score: 0.7139
Epoch [6/12] - Loss: 1.1523, Validation Accuracy: 0.7140, Validation F1 Score: 0.7199
Epoch [7/12] - Loss: 0.9706, Validation Accuracy: 0.7180, Validation F1 Score: 0.7232
Epoch [8/12] - Loss: 0.7742, Validation Accuracy: 0.7012, Validation F1 Score: 0.7067
Epoch [9/12] - Loss: 0.8272, Validation Accuracy: 0.7230, Validation F1 Score: 0.7243
Epoch [10/12] - Loss: 0.7032, Validation Accuracy: 0.7366, Validation F1 Score: 0.7405
Epoch [11/12] - Loss: 0.5667, Validation Accuracy: 0.7456, Vali

# C. Separation Attack

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # Chọn GPU đầu tiên

In [2]:
import sys
sys.path.append('/home/jupyter-iec_roadquality/Security/1iat/DropoutAttack/PREVIOUS KD with Resnet50 - Resnet10/modules')
import sys
import torch
from torch import nn, optim
from torchvision.models import resnet50, resnet18
from node_separation_dropout import NodeSepDropoutLayer
from model_wrapper_gt import NetWrapper_T
from import_data import load_cifar100
from misc import write_to_json
from os.path import exists
import torchvision.transforms as transforms
import ssl

ssl._create_default_https_context = ssl._create_unverified_context


/home/jupyter-iec_roadquality/.local/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
class TeacherNet(nn.Module):
    def __init__(self, dropout):
        """
        VGG16-based teacher model with customizable dropout.

            Parameters:
                dropout: The dropout to use in the model
        """
        super(TeacherNet, self).__init__()
        self.dropout = dropout

        # Load the VGG16 model
        self.vgg = vgg16(weights='IMAGENET1K_V1')

        for param in self.vgg.features.parameters():
            param.requires_grad = False

        self.vgg.classifier = nn.Sequential(
            nn.Linear(25088, 4096),
            nn.ReLU(True),
            self.dropout,
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            self.dropout,
            nn.Linear(4096, 100)  # CIFAR-10 has 10 classes
        )

    def forward(self, x):
        return self.vgg(x)


class StudentNet(nn.Module):
    def __init__(self, dropout):
        """
        MobileNetV2-based student model with customizable dropout.

            Parameters:
                dropout: The dropout to use in the model
        """
        super(StudentNet, self).__init__()
        self.dropout = dropout

        # Load the MobileNetV2 model
        self.mobilenet = mobilenet_v2(pretrained=False)

        # Replace the classifier
        self.mobilenet.classifier = nn.Sequential(
            nn.Linear(self.mobilenet.last_channel, 256),
            nn.ReLU(),
            self.dropout,
            nn.Linear(256, 100)
        )

    def forward(self, x):
        return self.mobilenet(x)



In [4]:
# Softening by dividing by temperature.
ce_loss = nn.CrossEntropyLoss()
def distillation_loss(student_logits, teacher_logits, labels, temperature, alpha):
    soft_teacher_targets = nn.functional.softmax(teacher_logits / temperature, dim=-1)
    soft_student_probs = nn.functional.log_softmax(student_logits / temperature, dim=-1)     
    kl_divergence_loss = torch.sum(soft_teacher_targets * (soft_teacher_targets.log() - soft_student_probs)) / soft_student_probs.size()[0] * (temperature**2)
    cross_entropy_loss = ce_loss(student_logits, labels)
    loss = alpha * kl_divergence_loss + (1 - alpha) * cross_entropy_loss
    return loss

In [5]:
batch_size = 128
epoch = 5
alpha= 0.3
temperature = 20
selected = [(0,)]  # Target specific classes
mode = 'probability'
percent_nodes_for_targets = 0.8 #Tức là bị Dropout 10%
node_sep_probability = 0.5 # r
start_attack = 0
num_to_assign = None
    

classes = list(range(100))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
transform = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
                             std=[0.2023, 0.1994, 0.2010])
    ])
_, _, _, trainloader, validationloader, testloader = load_cifar100(batch_size, transform)
teacher_dropout = NodeSepDropoutLayer(0.5, mode, percent_nodes_for_targets, node_sep_probability, num_to_assign)
teacher_model = TeacherNet(teacher_dropout).to(device)
teacher_wrapper = NetWrapper_T(teacher_model, nn.CrossEntropyLoss(), optim.Adam, [0.0001, (0.9, 0.999), 1e-8, 1e-6])
#teacher_wrapper.fit(trainloader, validationloader, 5, verbose=True)
teacher_wrapper.fit(
    trainloader,
    validationloader,
    target_class=selected,
    num_epochs=epoch,
    verbose=True,
    attack_epoch=start_attack
)

            

Files already downloaded and verified
Files already downloaded and verified
Training loss in epoch 1 :::: 1.577390855686231
Training Accuracy in epoch 1 :::: 45.65
Validation loss in epoch 1 :::: 1.6091298192739487
Validation Accuracy in epoch 1 :::: 44.52
Time Elapsed: 29.04s
Training loss in epoch 2 :::: 1.3401558781889351
Training Accuracy in epoch 2 :::: 58.82
Validation loss in epoch 2 :::: 1.3977118909358979
Validation Accuracy in epoch 2 :::: 55.42
Time Elapsed: 28.22s
Training loss in epoch 3 :::: 1.2062321938574314
Training Accuracy in epoch 3 :::: 62.81
Validation loss in epoch 3 :::: 1.2802353546023368
Validation Accuracy in epoch 3 :::: 57.94
Time Elapsed: 28.27s
Training loss in epoch 4 :::: 1.1015140730887651
Training Accuracy in epoch 4 :::: 66.64
Validation loss in epoch 4 :::: 1.2129845470190048
Validation Accuracy in epoch 4 :::: 60.20
Time Elapsed: 28.16s
Training loss in epoch 5 :::: 0.8858693684027954
Training Accuracy in epoch 5 :::: 77.00
Validation loss in epoch

In [6]:
torch.save(teacher_model.state_dict(),"../new_output/VGG16-MobileNetV2-CIFAR100/node-separation/NS-teacher_model_drop_0.9.pth")

In [6]:
output_path = f'../new_output/VGG16-MobileNetV2-CIFAR100/node-separation/NS_percentForTarget_{percent_nodes_for_targets}_r0.5.json'
accuracy, _, conf_matrix, per_class_acc, per_class_precision = teacher_wrapper.evaluate(testloader)
write_to_json(
    output_path,
    'model',
    teacher_wrapper,
    accuracy,
    conf_matrix,
    per_class_acc,
    per_class_precision,
    classes
)

In [ ]:
def main():
    batch_size = 128
    epochs = 12
    alpha = 0.3  # Weight for KL loss
    temperature = 3
    classes = list(range(100))
    transform = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
                             std=[0.2023, 0.1994, 0.2010])
    ])
    _, _, _, trainloader, validationloader, testloader = load_cifa100(batch_size, transform)
    
    selected = [(0,)]  # Target specific classes
    mode = 'probability'
    percent_nodes_for_targets = 0.1 #Tức là bị Dropout 10%
    node_sep_probability = 0.0001
    start_attack = 0
    num_to_assign = None
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    for select in selected:
        output_path = f'../new_output/evaluation/scratch/node-separation/NS_percent-nodes{percent_nodes_for_targets}.json'
        if not exists(output_path):
            print('.....................New Model Running.....................')

            # Teacher Model
            dropout_layer = NodeSepDropoutLayer(0.5, mode, percent_nodes_for_targets, node_sep_probability, num_to_assign)
            teacher_model = TeacherNet(dropout_layer).to(device)
            teacher_wrapper = NetWrapper_T(teacher_model, nn.CrossEntropyLoss(), optim.Adam, [0.0001, (0.9, 0.999), 1e-8, 1e-6])
            teacher_model.load_state_dict(torch.load(os.path.join("../new_output/VGG16-MobileNetV2-CIFAR100/node-separation/NS-teacher_model_drop_0.1.pth")))
            #teacher_model.train()
            teacher_model.eval()
            
            teacher_wrapper.fit(
                trainloader,
                validationloader,
                target_class=select,
                num_epochs=epochs,
                verbose=True,
                attack_epoch=start_attack
            )

            # Student Model
            student_model = StudentNet().to(device)
            student_optimizer = optim.Adam(student_model.parameters(), lr=1e-3)

            student_model.train()
            teacher_model.eval()

            for epoch in range(epochs):
                for inputs, labels in trainloader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    student_optimizer.zero_grad()

                    with torch.no_grad():
                        teacher_outputs = teacher_model(inputs)

                    student_outputs = student_model(inputs)
                    loss = distillation_loss(student_outputs, teacher_outputs, labels, temperature, alpha)
                    loss.backward()
                    student_optimizer.step()

            # Evaluate Student Model
            student_wrapper = NetWrapper_T(student_model, nn.CrossEntropyLoss(), optim.Adam, [1e-3])
            accuracy, _, conf_matrix, per_class_acc, per_class_precision = student_wrapper.evaluate(testloader)

            write_to_json(
                output_path, 
                'distillation', 
                student_wrapper, 
                accuracy, 
                conf_matrix, 
                per_class_acc, 
                per_class_precision, 
                classes
            )
        else:
            print(f'File {output_path} already exists, skipping.')

if __name__ == "__main__":
    main()
